## Pauli Transfer Matrix extraction
$\def\bra#1{\mathinner{\left\langle{#1}\right|}}\def\ket#1{\mathinner{\left|{#1}\right\rangle}}$

In [1]:
import numpy as np; pi = np.pi
import scipy.io as spio
from qutip import *

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# Figures should have a width of a 8.6 cm or 3 3/8 in, the width of a single manuscript column.
golden_mean = (np.sqrt(5)-1.0)/2.0 # Aesthetic ratio
fig_width = 3+3/8 # width in inches
fig_height = fig_width*golden_mean # height in inches (0.75 is the standard height to width ratio in Python)
fig_size = [fig_width, fig_height]
mpl.rcParams.update({
    'axes.labelsize' : 12,
    'font.size' : 12,
    'legend.fontsize' : 8,
    'xtick.labelsize' : 10,
    'ytick.labelsize' : 10,
    'ytick.minor.pad' : -0.5,
    'ytick.minor.pad' : -0.5,
    'ytick.major.size' : 1,
    'ytick.minor.size' : 1,
    'ytick.major.width' : .5,
    'ytick.minor.width' : .5,
    'xtick.major.size' : 1,
    'xtick.minor.size' : 1,
    'xtick.major.width' : .5,
    'xtick.minor.width' : .5,
    'xtick.major.pad' : 1.5,
    'xtick.minor.pad' : 1.5,
    'text.usetex' : True,
    'figure.dpi' : 100,
})

## Parameters

In [2]:
num_lvl = 20 # number of levels
a = destroy(num_lvl) # annihilation operator
K = 1 # kerr amplitude
G = 4*K # two photon pump amplitude
alpha = np.sqrt(G/K) # coherent state amplitude

# cat states
cat_plus = (coherent(num_lvl,alpha) + coherent(num_lvl,-alpha)).unit()
cat_minus = (coherent(num_lvl,alpha) - coherent(num_lvl,-alpha)).unit()

# computational basis
up = (cat_plus + cat_minus)/np.sqrt(2)
down = (cat_plus - cat_minus)/np.sqrt(2)

# Identity
I = up*up.dag() + down*down.dag()

# sigma-z in computational basis
sigma_z = up*up.dag() - down*down.dag()

# sigma-x in computational basis
sigma_x = up*down.dag() + down*up.dag()

# sigma-y in computational basis
sigma_y = 1j*(-up*down.dag() + down*up.dag())

# Array with Pauli matrices
P = [I, sigma_x, sigma_y, sigma_z]

## $R_z(\phi)$-gate

In [46]:
# gate time
T_g = 2/K

# single photon pump amplitude
def E(t,args):
    phi = args['phi']
    return np.pi*phi/(8*T_g*alpha)*np.sin(np.pi*t/T_g)

# Hamiltonian
H0 = - K * pow(a.dag(),2)*pow(a,2) + G * (pow(a.dag(),2) + pow(a,2))
H1 = a.dag() + a
H_tot = [H0,[H1,E]]

### quantum map

A quantum map $\Lambda$ is the following.
$$
    \Lambda
    = \begin{pmatrix}
        \Lambda'(\ket{0}\bra{0}) & \cdots & \Lambda'(\ket{0}\bra{20}) \\
        \vdots & \ddots & \vdots \\
        \Lambda'(\ket{20}\bra{0}) & \cdots & \Lambda'(\ket{20}\bra{20})
    \end{pmatrix}
$$
$\Lambda'(\ket{i}\bra{j})$ is $20\times 20$ matrix, so $\Lambda$ is $20^2\times 20^2$ matrix.

In [47]:
gamma = 1/1500 # single-photon loss rate
c_ops = np.sqrt(gamma)*a # collapse operator

# For precise calculation
opt = Options(nsteps=25000, atol=1e-10, rtol=1e-8)

# Quantum map
prop = propagator(H_tot, T_g, c_op_list = [c_ops], args = {'phi': pi/2}, options = opt)

### Kraus map
$$
\Lambda(\rho) = \sum_{k=1}^N A_k\rho A_k^\dagger
$$
where $A_k$ are Kraus-operators

In [48]:
kraus_form = to_kraus(prop)

### Pauli transfer matrix
$$
(R_\Lambda)_{ij} = \frac{1}{d}\mathrm{Tr}[P_i\Lambda(P_j)] = \frac{1}{d}\sum_{k=1}^N\mathrm{Tr}[P_iA_kP_jA_k^\dagger]
$$

In [ ]:
qsave("results/ ",prop)

In [49]:
d = 2
# pauli transfer matrix
R = np.zeros((d**2,d**2))
for i in range(d**2):
    for j in range(d**2):
        R[i,j] = 1/d * np.real(sum([(P[i]*A_k*P[j]*A_k.dag()).tr() for A_k in kraus_form]))
R = Qobj(R,dims=[[[2],[2]] for i in range(d)]) # Make quantum object

In [52]:
R.tidyup(1e-4)

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = False
Qobj data =
[[ 1.00000374  0.          0.          0.        ]
 [ 0.          0.         -0.98938706  0.        ]
 [ 0.          0.98939406  0.          0.        ]
 [ 0.          0.          0.          0.99999669]]

In [21]:
R.tidyup(1e-3)

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = False
Qobj data =
[[ 0.99999648  0.          0.          0.        ]
 [ 0.         -0.98938852  0.          0.        ]
 [ 0.          0.         -0.98938469  0.        ]
 [ 0.          0.          0.          0.99999253]]

In [35]:
ket0 = basis(2,0)
ket1 = basis(2,1)

In [44]:
rho = ket2dm(ket1)
rho

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 0.]
 [0. 1.]]

In [41]:
vec_rho = operator_to_vector(rho)

In [42]:
R*vec_rho

Quantum object: dims = [[[2], [2]], [1]], shape = (4, 1), type = operator-ket
Qobj data =
[[ 0.        ]
 [ 0.        ]
 [-0.01041129]
 [-0.9953308 ]]

## $R_x(\theta)$-gate

In [23]:
T_g = 10/K # gate time

# detuning
def Delta(t,args):
    Delta_0 = args['Delta_0']
    return Delta_0 * pow(np.sin(np.pi*t/T_g),2)

# Hamiltonian
H0 = - K * pow(a.dag(),2)*pow(a,2) + G * (pow(a.dag(),2) + pow(a,2))
H1 = - a.dag()*a
H_tot = [H0,[H1,Delta]]

In [27]:
gamma = 1/1500 # single-photon loss rate
c_ops = np.sqrt(gamma)*a # collapse operator

# For precise calculation
opt = Options(nsteps=45000, atol=1e-10, rtol=1e-8)

# Quantum map
prop = propagator(H_tot, T_g, c_op_list = [c_ops], args = {'Delta_0': 3.95}, options = opt)

KeyboardInterrupt: 

In [28]:
prop

Quantum object: dims = [[[20], [20]], [[20], [20]]], shape = (400, 400), type = super, isherm = False
Qobj data =
[[ 2.15440413e-01-2.23050785e-16j  0.00000000e+00+0.00000000e+00j
   1.51534681e-01+2.59015668e-01j ...  3.31125379e-07-5.10020304e-10j
   0.00000000e+00+0.00000000e+00j  7.83650581e-08-1.96311343e-20j]
 [ 0.00000000e+00+0.00000000e+00j -2.19821822e-01-9.44726167e-02j
   0.00000000e+00+0.00000000e+00j ...  0.00000000e+00+0.00000000e+00j
  -4.53472615e-10+3.23928168e-09j  0.00000000e+00+0.00000000e+00j]
 [ 1.51602911e-01+2.59014823e-01j  0.00000000e+00+0.00000000e+00j
   2.41962625e-01-2.95829185e-02j ... -6.26361518e-07-1.59056315e-09j
   0.00000000e+00+0.00000000e+00j -1.57398372e-07-4.53017915e-10j]
 ...
 [ 3.59611055e-08+4.14077659e-11j  0.00000000e+00+0.00000000e+00j
  -5.69885432e-08+3.66489446e-10j ...  6.70624736e-02-1.19962263e-01j
   0.00000000e+00+0.00000000e+00j -3.01130881e-01-5.17311844e-02j]
 [ 0.00000000e+00+0.00000000e+00j  7.85826335e-11+2.65020205e-09j
   